In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])


Bad key text.latex.preview in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\

In [2]:
import os
import pandas as pd
import numpy as np
from quetzal.model import stepmodel
from quetzal.io import excel
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex

In [26]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'
input_static_path = '../input_static/'
output_path = '../output/'

In [4]:
# Load parameters for settings
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

# Generation step: Trips per person

## Requires zones

## Saves generation volumes

In [5]:
# Choose purposes for which a logit model will be estimated
purposes = ['shopping', 'errands', 'leisure', 'accompany']

In [6]:
sm = stepmodel.read_json(model_path + 'de_zones')

In [7]:
# Load inter-zonal composite cost from mode choice step
cc = pd.read_csv(input_static_path + '/mode_choice_od_composite_cost.csv')
cc.set_index(['origin', 'destination'], inplace=True)
# Rename segments to integer values
cc.columns = pd.MultiIndex.from_tuples(
    [(seg.split('_')[0], {'no': 0, 'car': 1}[seg.split('_')[1]])
     for seg in cc.columns],
    names=['purpose_model', 'car_av'])
# Reshape the table into a mergable format
# Use the mean CC to all destinations as accessibility
cc = cc.unstack('destination').stack('purpose_model').stack('car_av').mean(axis=1)
cc = cc.unstack('purpose_model')

In [8]:
# Load estimation results
try:
    betas = pd.read_csv(input_path + 'generation_betas_{}.csv'.format(scenario), index_col=0)
except FileNotFoundError:
    betas = pd.read_csv(input_path + 'generation_betas.csv', index_col=0)

In [22]:
# Load base year values
base_data = pd.read_csv(input_path + 'generation_base_data.csv', index_col=[0,1])

In [24]:
# Define utility functions
Vs = {}

# Number of choices = max_trips-1 (because of 0 trips)
max_trips = 6
# create a database for biogeme functionality
for p in purposes:
    base_data['acc_'+p] = 0
database = db.Database('base', base_data)
globals().update(database.variables)
# And parameters
ASC = {0: ex.Beta('ASC_0', 0, None, None, 0)} # The only free one
ASC.update({j: ex.Beta('ASC_'+str(j), 0, None, None, 1) for j in range(1, max_trips)})
b_pop = {j: ex.Beta('b_pop_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_size = {j: ex.Beta('b_size_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_in = {j: ex.Beta('b_in_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_work = {j: ex.Beta('b_work_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_learn = {j: ex.Beta('b_learn_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_care = {j: ex.Beta('b_care_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_ticket = {j: ex.Beta('b_ticket_'+str(j), 0, None, None, 0) for j in range(max_trips)}
b_acc = {j: ex.Beta('b_acc_'+str(j), 0, None, None, 0) for j in range(max_trips)}

car_names = {0: '_no_car', 1: '_car'}
for car in [0,1]:
    Vs[car] = {}
    
    # Define the utility formulation by purpose
    '''Vs[car]['commuting'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         #+ hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_working * b_work[j]
         + pt_ticket * b_ticket[j]
         + acc_commuting * b_acc[j]
         for j in range(max_trips)}
    Vs[car]['business'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_working * b_work[j]
         + is_learning * b_learn[j]
         + is_caring * b_care[j]
         + pt_ticket * b_ticket[j]
         + acc_business * b_acc[j]
         for j in range(max_trips)}
    Vs[car]['education'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_learning * b_learn[j]
         + pt_ticket * b_ticket[j]
         + acc_education * b_acc[j]
         for j in range(max_trips)}'''
    Vs[car]['shopping'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_working * b_work[j]
         + is_caring * b_care[j]
         #+ pt_ticket * b_ticket[j]
         + acc_shopping * b_acc[j]
         for j in range(max_trips)}
    Vs[car]['errands'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_caring * b_care[j]
         #+ pt_ticket * b_ticket[j]
         + acc_errands * b_acc[j]
         for j in range(max_trips)}
    Vs[car]['leisure'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_working * b_work[j]
         + is_learning * b_learn[j]
         + is_caring * b_care[j]
         #+ pt_ticket * b_ticket[j]
         + acc_leisure * b_acc[j]
         for j in range(max_trips)}
    Vs[car]['accompany'] = {j:
         ASC[j]
         + ex.log(1+population)*b_pop[j]
         + hh_size * b_size[j]
         + hh_income * b_in[j]
         + is_learning * b_learn[j]
         + is_caring * b_care[j]
         + pt_ticket * b_ticket[j]
         + acc_accompany * b_acc[j]
         for j in range(max_trips)}

In [27]:
# Retrieve the household size change
# based on data from the "BBSR Raumordnungsprognose 2040"
hh_data = pd.read_excel(input_static_path + 'spatial_households_2040.xlsx',
                        sheet_name='Bev_Typ_g_17-40', usecols=list(range(1,9)), skiprows=3)
year = min(int(params['general']['year']), 2040)
year_rounded = int(5 * round(year / 5))
# Get average hh size for an urbanisation category
def get_av_hh_size(group, y):
    weighted_sum = 0
    for size, weight in zip([1,2,3,4,5], [1,1,1,0.63,0.37]):
        weighted_sum += group.set_index('HH-Größe').loc[size, y] * size * weight
    hh_sum = group.set_index('HH-Größe').loc['alle', y]
    return weighted_sum / hh_sum
# Compute changes in hh size
hh_size_change = (hh_data.groupby('Kennziffer').apply(get_av_hh_size, year_rounded)
                  - hh_data.groupby('Kennziffer').apply(get_av_hh_size, 2017)) \
                 / hh_data.groupby('Kennziffer').apply(get_av_hh_size, 2017) + 1 \
                 if year != 2017 else {1:1, 2:1, 3:1}

## Calculate generation volumes in model zones

In [28]:
sm.zones['population'].sum() / 1e6

83.146029

In [29]:
# Construct a database with one entry per zone
# for simulation of trip generation probabilities
sim_zones = [pd.DataFrame()]
for u in [1,2,3]:
    means = base_data.xs(u, level='urbanisation')
    mask = sm.zones['urbanisation']==u
    sim_zones.append(pd.DataFrame(index=sm.zones.loc[mask].index,
                                  data={
        'population': sm.zones.loc[mask, 'population'],
        'hh_size': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['hh_size']) * hh_size_change[u],
        'hh_income': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['hh_income']) * params['income_change'][u],
        'is_working': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['is_working']) * params['employed_change'][u],
        'is_learning': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['is_learning']),
        'is_caring': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['is_caring']),
        'pt_ticket': sm.zones.loc[mask, 'NUTS_ID'].str[:3].map(means['pt_ticket'])
    }))
sim_zones = pd.concat(sim_zones)

In [30]:
# Simulate the probabilities and put them into the zones table
for p in purposes:
    for car in [0,1]:
        # Add accessibility for this segment to database
        sim_zones['acc_'+p] = sim_zones.merge(cc.xs(car, level='car_av'),
                                              how='left', left_index=True, right_index=True
                                             )[p]
        zones_db = db.Database('zones', sim_zones.loc[~sim_zones.isna().any(axis=1)])
        
        # Simulate
        simulate = {j: models.logit(Vs[car][p], None, j) for j in range(max_trips)}
        model = bio.BIOGEME(zones_db, simulate, numberOfThreads=1)
        model.generateHtml = False
        model.generatePickle = False
        model.saveIterations = False
        probs = model.simulate(theBetaValues=betas[p+car_names[car]].to_dict())
        
        # Calculate generation volumes
        trips_here = 1 * probs[1] + 2 * probs[2] + 3 * probs[3] + 4 * probs[4] + 5 * probs[5]
        print('{}_{}: {} trips p.p.'.format(p,car,np.round(trips_here.mean(), 2)))
        if car == 0:
            sm.zones[p+car_names[car]] = \
                trips_here * sm.zones['population'] * (1 - sm.zones['car_avail_hh']) * 365
        else:
            sm.zones[p+car_names[car]] = \
                trips_here * sm.zones['population'] * sm.zones['car_avail_hh'] * 365

buy/execute_0: 0.33 trips p.p.
buy/execute_1: 0.48 trips p.p.
leisure_0: 0.41 trips p.p.
leisure_1: 0.43 trips p.p.
accompany_0: 0.08 trips p.p.
accompany_1: 0.14 trips p.p.


In [31]:
columns = [p+car_names[c] for p in purposes for c in [0,1]]
print("Sum of trips per day with model population (million): {}".format(
    sm.zones[columns].sum().sum() / 1e6 / 365))

Sum of trips per day with model population (million): 85.10459989815276


## Save results

In [32]:
sm.zones.loc[~sm.zones[columns].isna().any(axis=1), columns].to_csv(
    model_path + 'generation_volumes.csv')

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 33/33 [00:08<00:00,  3.80it/s]
